In [0]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [1]:
from google.colab import files
uploaded = files.upload()

Saving smile-annotations-final.csv to smile-annotations-final.csv


In [0]:
import pandas as pd

df = pd.read_csv('smile-annotations-final.csv', names= ['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
df.text.iloc[1]

'Dorian Gray with Rainbow Scarf #LoveWins (from @britishmuseum http://t.co/Q4XSwL0esu) http://t.co/h0evbTBWRq'

In [7]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [0]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [9]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [0]:
possible_labels = df.category.unique()

In [0]:
label_dict = {}
for index , possible_label in enumerate(possible_labels) :
    label_dict[possible_label] = index

In [12]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [0]:
df['label'] = df.category.replace(label_dict)

In [14]:
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)
  

In [0]:
df['data_type'] = ['not_set']*df.shape[0]

In [0]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [20]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [22]:
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 13.5MB/s 
     |████████████████████████████████| 890kB 27.3MB/s 
     |████████████████████████████████| 1.1MB 39.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9a2cf1749e2c676e1f683325cd48986ec2199d78ec8d522395a6bbbfae91aa4c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [24]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [0]:
import torch

from tqdm.notebook import tqdm
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)



input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)


input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [0]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train, labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val, labels_val)

In [30]:
len(dataset_train), len(dataset_val)

(1258, 223)

In [0]:
from transformers import BertForSequenceClassification

In [32]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [0]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)


dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

In [0]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [0]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [0]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

In [0]:
import numpy as np

In [0]:
from sklearn.metrics import f1_score

In [0]:
def f1_score_func(preds, labels) :
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')


In [0]:
def accuracy_per_class(preds, labels) :
  label_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
      



In [0]:
import random
import torch
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [0]:
def evaluate(dataloader_val):

  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':   batch[0],
              'attention_mask': batch[1],
              'labels':         batch[2],
             }
  with torch.no_grad():
      outputs=model(**inputs)


      loss=outputs[0]
      logits=outputs[1]
      loss_val_total += loss.item()


      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

      loss_val_avg = loss_val_total/len(dataloader_val)


      predictions = np.concatenate(predictions, axis=0)
      true_vals = np.concatenate(true_vals, axis=0)

      return loss_val_avg, predictions, true_vals


In [69]:
from tqdm.notebook import tqdm 
      
for epoch in tqdm(range(1, epochs+1)):



  model.train()

  loss_train_total = 0 
  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False) 
  for batch in progress_bar:

    model.zero_grad()
 
    batch = tuple(b.to(device) for b in batch)
 
    inputs = {'input_ids':   batch[0],
          'attention_mask': batch[1],
          'labels':         batch[2],
         }
  outputs = model(**inputs)
 
  loss=outputs[0]
  loss_train_total += loss.item()
  loss.backward()
 
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
 
  optimizer.step()
  scheduler.step()
 
  progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
 
 
 
  tqdm.write('\n Epoch {epoch}')
 
  loss_train_avg = loss_train_total/len(dataloader_train)
 
  tqdm.write(f'Training Loss: {loss_train_avg}')
 
  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Vlidation loss: {val_loss}')
  tqdm.write(f'F1 Score (weighted): {val_f1}')



 Epoch {epoch}
Training Loss: 0.0020718593446035233
Vlidation loss: 0.021465605923107693
F1 Score (weighted): 0.5333333333333333



 Epoch {epoch}
Training Loss: 0.002227004369099935
Vlidation loss: 0.024383708834648132
F1 Score (weighted): 0.5333333333333333



 Epoch {epoch}
Training Loss: 0.004750990489172557
Vlidation loss: 0.011099654648985182
F1 Score (weighted): 1.0



 Epoch {epoch}
Training Loss: 0.0021137922529190305
Vlidation loss: 0.010326720774173737
F1 Score (weighted): 1.0



 Epoch {epoch}
Training Loss: 0.0022800388790312266
Vlidation loss: 0.023156525833266123
F1 Score (weighted): 0.5333333333333333



 Epoch {epoch}
Training Loss: 0.0055462625291612415
Vlidation loss: 0.009255800928388323
F1 Score (weighted): 1.0



 Epoch {epoch}
Training Loss: 0.0019780510947817846
Vlidation loss: 0.02096383912222726
F1 Score (weighted): 0.5333333333333333



 Epoch {epoch}
Training Loss: 0.004678620232476128
Vlidation loss: 0.01965829304286412
F1 Score (weighted): 0.5333333333333333



 Epoch {epoch}
Training Loss: 0.004179064432779948
Vlidation loss: 0.009341202676296234
F1 Score (weighted): 1.0



 Epoch {epoch}
Training Loss: 0.0019176248520139665
Vlidation loss: 0.022452098982674733
F1 Score (weighted): 0.5333333333333333



In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [0]:
model.to(device)
pass

In [0]:
_, predictions, true_vals = evaluate(dataloader_val)

In [90]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 0/3
